In [ ]:
# 1. Katie Corcoran

In [1]:
# 2.
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

path = '/Users/katiecorcoran/Dropbox/5330corgis'
files = os.listdir(path)
for file in files:
    print(file)

pdx623.csv
portlandeventjun22.csv
newreg-062822.csv
Icon
newcorgis-20220619.csv
newcorgis-20220618.csv
signupsjune272022.csv
tacoma-062322.csv


In [3]:
# 3.
df = pd.read_csv(f'{path}/pdx623.csv')
print(df)

                            Name Breed Gender  Weight  Age
0  Wendell of the Gnashing Teeth   CWC     NM      32  3.3
1                Trotsky the Dog   Pem     SF      23  5.5
2                CanCan of Beans   Pem     SF      25  4.2
3             Harvey the Younger   PWC      M      22  1.1
4                 Jenni the Shiv   PWC     SF      28  3.0
5           Ogbert the Semi-Wise   Pem     NM      25  4.0
6                Jeff OCorgosaur   PWC      M      27  2.0
7                  Poofy Mudfoot   Pem     SF      23  2.1


In [7]:
# 4.
import psycopg2

conn = psycopg2.connect(
database='corgirace',
user='USER',
password='PASSWORD',
host='HOST',
port= '5432')
cursor = conn.cursor()

for file in files:
    # skip if file is not csv
    if file[-3:] != 'csv':
        print(f'Skipped file {file}')
        continue
    
    # check if file already exists
    cursor.execute(f'''
        SELECT
            COUNT(*)
        FROM ingest
        WHERE filename = '{file}';
    ''')
    y = cursor.fetchone()
    if y[0] > 0:
        print(f'File {file} has already been processed')
        continue
    
    # process the file
    print(f'File {file} needs to be processed.')
    df = pd.read_csv(f'{path}/{file}')
    # ensure the data is in the right format
    df['Gender'].replace(to_replace='Fem', value='F', inplace=True)
    df['Breed'].replace(to_replace=['PWC', 'CWC'], value=['Pem', 'Cardi'], inplace=True)
    df['Weight'] = round(df['Weight'])
    
    # insert into database
    for x in df.index:
        cursor.execute(f'''
            INSERT INTO corgi (corgname, breed, gender, weight, age, fromfile)
            VALUES (
            '{df.loc[x]['Name']}',
            '{df.loc[x]['Breed']}',
            '{df.loc[x]['Gender']}',
            {df.loc[x]['Weight']},
            {df.loc[x]['Age']},
            '{file}');
        ''')
        conn.commit()
        print(f"Record created for {df.loc[x]['Name']}.")
        
    # insert into ingest table
    cursor.execute(f'''
        INSERT INTO ingest (filename, whendone)
        VALUES ('{file}', CURRENT_TIMESTAMP);
    ''')
    conn.commit()

File pdx623.csv needs to be processed.
Record created for Wendell of the Gnashing Teeth.
Record created for Trotsky the Dog.
Record created for CanCan of Beans.
Record created for Harvey the Younger.
Record created for Jenni the Shiv.
Record created for Ogbert the Semi-Wise.
Record created for Jeff OCorgosaur.
Record created for Poofy Mudfoot.
File portlandeventjun22.csv needs to be processed.
Record created for Amber Waves of Fur.
Record created for Kodlak Whitemane.
Record created for Floaty Bee.
Record created for Bag o Donuts.
Record created for Hot Ham and Cheese.
Record created for Cruzer the Cruzedog.
Record created for Sally the Destroyer.
Record created for Elma Redfur.
Record created for Kool Thing.
Record created for Skitters McDogdog.
Record created for Ophelio.
Record created for Mr. Stealyoursandwich.
File newreg-062822.csv needs to be processed.
Record created for Megalodon.
Record created for Sparky Punkbottom.
Record created for Esme the Pointy-Eared.
Record created fo

In [9]:
# 5.
df = pd.read_sql('''
SELECT
    corgi.corgid,
    corgi.corgname,
    ingest.filename,
    DATE(ingest.whendone)
FROM corgi
JOIN ingest ON ingest.filename = corgi.fromfile
WHERE DATE_TRUNC('month', ingest.whendone) = DATE_TRUNC('month', CURRENT_TIMESTAMP)
ORDER BY corgi.corgname;
''', conn, index_col='corgid')
print(df)

                             corgname                filename        date
corgid                                                                   
81                 Amber Waves of Fur  portlandeventjun22.csv  2024-10-15
84                       Bag o Donuts  portlandeventjun22.csv  2024-10-15
97                Betty Doggy Sprinty       newreg-062822.csv  2024-10-15
112                        Briar Rose       tacoma-062322.csv  2024-10-15
75                    CanCan of Beans              pdx623.csv  2024-10-15
103                        Cashew Nut   signupsjune272022.csv  2024-10-15
111                         Cleopupra       tacoma-062322.csv  2024-10-15
108                    Courtesy Check   signupsjune272022.csv  2024-10-15
86                Cruzer the Cruzedog  portlandeventjun22.csv  2024-10-15
109                            Dogpup       tacoma-062322.csv  2024-10-15
88                        Elma Redfur  portlandeventjun22.csv  2024-10-15
95              Esme the Pointy-Eared 

/var/folders/44/s9hsvm1943z9xb042mqxxtgc0000gn/T/ipykernel_14745/1021812437.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('''
